In [1]:
import tensorflow as tf
import numpy as np
import cv2

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path=r'E:\jupyter notebook\3.tflite')
interpreter.allocate_tensors()

# Define the edges (connections between keypoints)
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

# Function to draw keypoints
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

# Function to draw connections
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

# Function to process video
def process_video(input_video_path, output_video_path):
    # Open the video file
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Reshape image
        img = frame.copy()
        img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
        input_image = tf.cast(img, dtype=tf.float32)
        
        # Setup input and output 
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        
        # Make predictions 
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
        
        # Rendering 
        draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
        draw_keypoints(frame, keypoints_with_scores, 0.4)
        
        # Write the frame into the output video file
        out.write(frame)
        
        # Display the frame (optional)
        cv2.imshow('Processed Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release everything when the job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
input_video = r'E:\CapStone\inputviedos\test8.mp4'
output_video = r'E:\CapStone\outputvideo\output8.mp4'
process_video(input_video, output_video)

In [8]:
import tensorflow as tf
import numpy as np
import cv2

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path=r'E:\jupyter notebook\3.tflite')
interpreter.allocate_tensors()

# Define the edges (connections between keypoints)
EDGES = {
    (0, 1): 'm', (0, 2): 'c', (1, 3): 'm', (2, 4): 'c', (0, 5): 'm', (0, 6): 'c',
    (5, 7): 'm', (7, 9): 'm', (6, 8): 'c', (8, 10): 'c', (5, 6): 'y', (5, 11): 'm',
    (6, 12): 'c', (11, 12): 'y', (11, 13): 'm', (13, 15): 'm', (12, 14): 'c', (14, 16): 'c'
}

# Function to draw keypoints
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0, 255, 0), -1)

# Function to draw connections
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

# Function to process a single image
def process_image(input_image_path, output_image_path):
    image = cv2.imread(input_image_path)
    if image is None:
        print("Error: Could not read image.")
        return
    
    img = image.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    draw_connections(image, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(image, keypoints_with_scores, 0.4)
    
    cv2.imwrite(output_image_path, image)
    print(f"Output saved to {output_image_path}")

# Example usage
input_image = r'E:\Fall Detection Dataset\fall_cam1\datasets\fall-01-cam0-rgb\fall-01-cam0-rgb-102.png'
output_image = r'E:\CapStone\output\f5.png'
process_image(input_image, output_image)


Output saved to E:\CapStone\output\f5.png


In [12]:
import tensorflow as tf
import numpy as np
import cv2
import os

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path=r'E:\jupyter notebook\3.tflite')
interpreter.allocate_tensors()

# Define the edges (connections between keypoints)
EDGES = {
    (0, 1): 'm', (0, 2): 'c', (1, 3): 'm', (2, 4): 'c', (0, 5): 'm', (0, 6): 'c',
    (5, 7): 'm', (7, 9): 'm', (6, 8): 'c', (8, 10): 'c', (5, 6): 'y', (5, 11): 'm',
    (6, 12): 'c', (11, 12): 'y', (11, 13): 'm', (13, 15): 'm', (12, 14): 'c', (14, 16): 'c'
}

# Function to draw keypoints
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0, 255, 0), -1)

# Function to draw connections
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

# Function to process all images in a folder
def process_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_image_path = os.path.join(input_folder, filename)
            output_image_path = os.path.join(output_folder, filename)
            
            image = cv2.imread(input_image_path)
            if image is None:
                print(f"Error: Could not read image {filename}.")
                continue
            
            img = image.copy()
            img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
            input_image = tf.cast(img, dtype=tf.float32)
            
            input_details = interpreter.get_input_details()
            output_details = interpreter.get_output_details()
            
            interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
            interpreter.invoke()
            keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
            
            draw_connections(image, keypoints_with_scores, EDGES, 0.4)
            draw_keypoints(image, keypoints_with_scores, 0.4)
            
            cv2.imwrite(output_image_path, image)
            print(f"Output saved to {output_image_path}")

# Example usage
input_folder = r'E:\Fall Detection Dataset\nonfall_cam1\nfDatasets\adl-40-cam0-rgb'
output_folder = r'E:\CapStone\output'
process_images(input_folder, output_folder)


Output saved to E:\CapStone\output\adl-40-cam0-rgb-001.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-002.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-003.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-004.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-005.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-006.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-007.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-008.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-009.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-010.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-011.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-012.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-013.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-014.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-015.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-016.png
Output saved to E:\CapStone\output\adl-40-cam0-rgb-017.p